In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import pyexasol

# fold

In [2]:
raw_fold = 'D:/data/processing/urea/raw'
df_fold = 'D:/data/processing/urea/df'
an_fold = 'D:/data/processing/urea/an'
folds = {
    'raw_fold':raw_fold, 
    'df_fold':df_fold, 
    'an_fold':an_fold, 
}
folds

{'raw_fold': 'D:/data/processing/urea/raw',
 'df_fold': 'D:/data/processing/urea/df',
 'an_fold': 'D:/data/processing/urea/an'}

# server

In [3]:
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')

# load

## 등록&제원

In [178]:
# 1m 10.8s
name = '경유_경유하이브리드_20231206'
file_name = f'{name}.csv'
cs_dgl = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, usecols=['VIN', 'EXHST_GAS_CERT_NO', 'EXHST_GAS_GRD_CD', 'YRIDNW', 'VHCTY_CD_x', 'FUEL_CD', 'VHCL_ERSR_YN', 'FRST_REG_YMD', 'MNFCTR_NM', 'EGIN_TY'])
# cs_dgl = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, nrows=10)
cs_dgl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11905778 entries, 0 to 11905777
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   EXHST_GAS_CERT_NO  object
 1   EXHST_GAS_GRD_CD   object
 2   FRST_REG_YMD       object
 3   VHCL_ERSR_YN       object
 4   YRIDNW             int64 
 5   VHCTY_CD_x         object
 6   VIN                object
 7   EGIN_TY            object
 8   FUEL_CD            object
 9   MNFCTR_NM          object
dtypes: int64(1), object(9)
memory usage: 908.3+ MB


In [179]:
ch_col = {
    'VIN':'차대번호', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'YRIDNW':'차량연식', 
    'VHCTY_CD_x':'차종', 
    'FUEL_CD':'연료',
    'VHCL_ERSR_YN':'차량말소유무',
    'FRST_REG_YMD':'최초등록일자',
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'MNFCTR_NM':'제작사명', 
    'EGIN_TY':'엔진형식', 
    # 'BSPL_STDG_CD':'법정동코드', 
    # 'MANP_MNG_NO':'제원관리번호', 
    # 'PURPS_CD2':'용도', 
    # 'VHCL_FBCTN_YMD':'제작일자', 
    # 'VHCL_MNG_NO':'차량관리번호', 
    # 'VHRNO':'자동차등록번호',
    # 'EXTGAS_INSP_VLD_YMD':'검사유효일',
    # 'VHCL_OWNR_CL_CD':'소유자구분',  
}
cs_dglr = cs_dgl.rename(columns=ch_col)
cs_dglr.shape

(11905778, 10)

In [180]:
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
cs_dglr['차종'] = cs_dglr['차종'].replace(cd_dict)
cs_dglr['차종'].unique()

array(['화물', '승용', '승합', '특수'], dtype=object)

In [181]:
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
cs_dglr['연료'] = cs_dglr['연료'].replace(fuel_dict)
cs_dglr['연료'].unique()

array(['경유', '경유 하이브리드'], dtype=object)

In [182]:
cs_dglr['연료'].value_counts(dropna=False)

연료
경유          11853965
경유 하이브리드       51813
Name: count, dtype: int64

In [183]:
cs_dglr['차량말소유무'].value_counts(dropna=False)

차량말소유무
N    9582519
Y    2323259
Name: count, dtype: int64

In [184]:
cs_dglr.groupby(['연료', '차량말소유무'], dropna=False)['차대번호'].count().reset_index()

,연료,차량말소유무,차대번호
0,경유,N,9533811
1,경유,Y,2320154
2,경유 하이브리드,N,48708
3,경유 하이브리드,Y,3105


In [185]:
cs = cs_dglr[(cs_dglr['연료'] == '경유') & (cs_dglr['차량말소유무'] == 'N')].reset_index(drop=True)
cs.shape

(9533811, 10)

In [186]:
cs['최초등록일자'].dtype

dtype('O')

In [187]:
cs['최초등록일자'] = pd.to_numeric(cs['최초등록일자'], errors='coerce')
cs['최초등록일자'].dtype

dtype('float64')

In [188]:
csm = cs.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
csm.shape

(9533740, 10)

## 저감장치 부착이력

In [131]:
# 3.0s
att = wd.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분',
    # 'RDCDVC_KND_CD':'저감장치종류',
}
attr = att.rename(columns=att_ch_col)
print(attr.columns)
print('data load : STD_DLM_TB_ERP_ATT_HIS')

Index(['차대번호', '저감장치구분'], dtype='object')
data load : STD_DLM_TB_ERP_ATT_HIS


In [132]:
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '2종', '엔진교체', '삼원촉매', '1종+SCR'], dtype=object)

In [133]:
attr.shape, len(attr['차대번호'].unique())

((1015941, 2), 1014369)

In [134]:
sorted(['1종', '3종', '엔진개조', '2종', '엔진교체', '삼원촉매', '1종+SCR'])

['1종', '1종+SCR', '2종', '3종', '삼원촉매', '엔진개조', '엔진교체']

In [135]:
# 차대번호 중복 제거
attm = attr.sort_values('저감장치구분').drop_duplicates('차대번호').reset_index(drop=True)
attm.shape

(1014369, 2)

## 인증대장

In [136]:
name = '인증대장(대형)_SCR_적용차종(EURO 4, 5)'
file_name = f'{name}.xlsx'
l45 = pd.read_excel(os.path.join(raw_fold, file_name))
l45.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   제작사         42 non-null     object 
 1   차명          42 non-null     object 
 2   자동차형식       42 non-null     object 
 3   엔진형식        42 non-null     object 
 4   배기량         42 non-null     float64
 5   최대출력        43 non-null     object 
 6   Unnamed: 6  43 non-null     object 
 7   변속기         42 non-null     object 
 8   총중량         42 non-null     object 
 9   공차중량        42 non-null     object 
 10  제작사.1       42 non-null     object 
 11  제작국         42 non-null     object 
 12  인증번호        43 non-null     object 
 13  인증일자        43 non-null     object 
 14  비고          42 non-null     object 
dtypes: float64(1), object(14)
memory usage: 5.2+ KB


In [137]:
l45_col = [
    '제작사1',
    '차명',
    '자동차형식',
    '엔진형식',
    '배기량',
    '최대출력_PS',
    '최대출력_RPM',
    '변속기',
    '총중량',
    '공차중량',
    '제작사2',
    '제작국',
    '인증번호',
    '인증일자',
    '비고',
]
l45.columns = l45_col
l45.columns

Index(['제작사1', '차명', '자동차형식', '엔진형식', '배기량', '최대출력_PS', '최대출력_RPM', '변속기',
       '총중량', '공차중량', '제작사2', '제작국', '인증번호', '인증일자', '비고'],
      dtype='object')

In [138]:
l45.shape

(43, 15)

In [139]:
l45.head()

,제작사1,차명,자동차형식,엔진형식,배기량,최대출력_PS,최대출력_RPM,변속기,총중량,공차중량,제작사2,제작국,인증번호,인증일자,비고
0,NaN,NaN,NaN,NaN,NaN,PS,RPM,NaN,NaN,NaN,NaN,NaN,배출,배출,NaN
1,현대자동차,현대25.5톤덤프트럭,HD255-YDU0-DHECA,D6CE,12742.0,520,1700,자동12단,38730,13230,현대자동차,한국,AMY-HD-14-35,2010-07-19 00:00:00,유로5
2,타타대우,대우25톤장축카고트럭,BT9CC1,DV11,10964.0,440,1900,수동16단,38800,13670,타타대우상용차,한국,7MY-DC-23-70,2007-12-18 00:00:00,유로4
3,타타대우,대우5톤초장축카고트럭,BC4AH1,F4AE3681D,5880.0,251,2700,수동6단,11415,6285,타타대우상용차,한국,7MY-DC-23-72,2007-12-18 00:00:00,유로4
4,타타대우,대우5톤초장축카고트럭,BC4AT1,DL06,5890.0,270,2500,수동6단,11500,6370,타타대우상용차,한국,8MY-DC-24-12,2008-05-21 00:00:00,유로4


In [140]:
l45 = l45.iloc[1:].reset_index(drop=True)
l45.head()

,제작사1,차명,자동차형식,엔진형식,배기량,최대출력_PS,최대출력_RPM,변속기,총중량,공차중량,제작사2,제작국,인증번호,인증일자,비고
0,현대자동차,현대25.5톤덤프트럭,HD255-YDU0-DHECA,D6CE,12742.0,520,1700,자동12단,38730,13230,현대자동차,한국,AMY-HD-14-35,2010-07-19 00:00:00,유로5
1,타타대우,대우25톤장축카고트럭,BT9CC1,DV11,10964.0,440,1900,수동16단,38800,13670,타타대우상용차,한국,7MY-DC-23-70,2007-12-18 00:00:00,유로4
2,타타대우,대우5톤초장축카고트럭,BC4AH1,F4AE3681D,5880.0,251,2700,수동6단,11415,6285,타타대우상용차,한국,7MY-DC-23-72,2007-12-18 00:00:00,유로4
3,타타대우,대우5톤초장축카고트럭,BC4AT1,DL06,5890.0,270,2500,수동6단,11500,6370,타타대우상용차,한국,8MY-DC-24-12,2008-05-21 00:00:00,유로4
4,타타대우,대우트랙터,EX3TX2,F3BE3681B,12882.0,560,1900,자동12단,26020,8810,타타대우상용차,한국,8MY-DC-24-15,2008-06-30 00:00:00,유로4


In [141]:
name = '인증대장(대형)_SCR_적용차종(EURO 6)'
file_name = f'{name}.xlsx'
l6 = pd.read_excel(os.path.join(raw_fold, file_name))
l6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   No           21 non-null     int64 
 1   제작사          21 non-null     object
 2   인증번호         21 non-null     object
 3   인증일자         21 non-null     object
 4   차명           21 non-null     object
 5   차종           21 non-null     object
 6   차형식          21 non-null     object
 7   엔진형식         21 non-null     object
 8   배기량          21 non-null     int64 
 9   변속기          21 non-null     object
 10  최대출력         21 non-null     int64 
 11  Unnamed: 11  21 non-null     int64 
 12  총중량          21 non-null     object
 13  공차중량         21 non-null     object
 14  제작국          21 non-null     object
 15  사용연료         21 non-null     object
 16  배출기준         21 non-null     object
 17  유럽기준         21 non-null     object
 18  SCR유무        21 non-null     object
dtypes: int64(4), object(15)
memory 

In [142]:
name = '인증대장(소형)_SCR_적용차종(EURO 6)'
file_name = f'{name}.xlsx'
s6 = pd.read_excel(os.path.join(raw_fold, file_name))
s6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   일련번호          47 non-null     int64 
 1   제작사           47 non-null     object
 2   배출가스차종        47 non-null     object
 3   배출가스인증번호      47 non-null     object
 4   배출가스인증일       47 non-null     object
 5   차명            47 non-null     object
 6   자동차형식         47 non-null     object
 7   엔진형식          47 non-null     object
 8   배기량           47 non-null     int64 
 9   변속기           47 non-null     object
 10  최대출력PS        47 non-null     int64 
 11  최대출력RPM       47 non-null     int64 
 12  총중량           47 non-null     int64 
 13  공차중량          47 non-null     int64 
 14  제작국가          47 non-null     object
 15  사용연료          47 non-null     object
 16  배출가스기준        47 non-null     object
 17  유럽 기준         47 non-null     object
 18  SCR 유(Y)무(N)  47 non-null     object
dtypes: int64(6

In [143]:
# 확인된 사항
# l45, l6, s6 배출가스인증번호(인증번호) 중복 없음

In [144]:
len(set(l45['인증번호'].to_list()))

42

In [145]:
len(set(l45['인증번호'].to_list()) - set(l6['인증번호'].to_list()))

42

In [146]:
len(set(l6['인증번호'].to_list()))

21

In [147]:
len(set(l6['인증번호'].to_list()) - set(s6['배출가스인증번호'].to_list()))

21

In [148]:
len(set(l45['인증번호'].to_list()) - set(s6['배출가스인증번호'].to_list()))

42

## 제작사 확인

In [205]:
name = '엔진형식_제작사명_리스트(수정)'
file_name = f'{name}.xlsx'
ck = pd.read_excel(os.path.join(raw_fold, file_name))
ck.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8679 entries, 0 to 8678
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   엔진형식                 8679 non-null   object
 1   제작사명                 8679 non-null   object
 2   차대번호                 8679 non-null   int64 
 3   국내/외 구분(1:국내, 2:국외)  8679 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 271.3+ KB


In [206]:
ck.head()

,엔진형식,제작사명,차대번호,"국내/외 구분(1:국내, 2:국외)"
0,-,(주) 두성특장차,1,1
1,-,한국특장차(주),1,1
2,.D4GB,(주) 골드밴,1,1
3,0,SOFA차량,4,1
4,0,남도수입,1,2


In [210]:
ck = ck.rename(columns={'국내/외 구분(1:국내, 2:국외)':'구분'})
ck['구분'] = ck['구분'].replace({1:'국내', 2:'국외'})
ck.head()

,엔진형식,제작사명,차대번호,구분
0,-,(주) 두성특장차,1,국내
1,-,한국특장차(주),1,국내
2,.D4GB,(주) 골드밴,1,국내
3,0,SOFA차량,4,국내
4,0,남도수입,1,국외


In [214]:
ck = ck.drop(['차대번호'], axis=1)
ck.head()

,엔진형식,제작사명,구분
0,-,(주) 두성특장차,국내
1,-,한국특장차(주),국내
2,.D4GB,(주) 골드밴,국내
3,0,SOFA차량,국내
4,0,남도수입,국외


In [216]:
ck.shape, ck.drop_duplicates(['엔진형식', '제작사명']).shape

((8679, 3), (8679, 3))

# 전처리
- 컬럼명 수정
- 필수 컬럼만 추출
- 등록 + 인증대장 + 저감

## 인증대장 병합

In [149]:
l45.columns.to_list()

['제작사1',
 '차명',
 '자동차형식',
 '엔진형식',
 '배기량',
 '최대출력_PS',
 '최대출력_RPM',
 '변속기',
 '총중량',
 '공차중량',
 '제작사2',
 '제작국',
 '인증번호',
 '인증일자',
 '비고']

In [150]:
l45.head()

,제작사1,차명,자동차형식,엔진형식,배기량,최대출력_PS,최대출력_RPM,변속기,총중량,공차중량,제작사2,제작국,인증번호,인증일자,비고
0,현대자동차,현대25.5톤덤프트럭,HD255-YDU0-DHECA,D6CE,12742.0,520,1700,자동12단,38730,13230,현대자동차,한국,AMY-HD-14-35,2010-07-19 00:00:00,유로5
1,타타대우,대우25톤장축카고트럭,BT9CC1,DV11,10964.0,440,1900,수동16단,38800,13670,타타대우상용차,한국,7MY-DC-23-70,2007-12-18 00:00:00,유로4
2,타타대우,대우5톤초장축카고트럭,BC4AH1,F4AE3681D,5880.0,251,2700,수동6단,11415,6285,타타대우상용차,한국,7MY-DC-23-72,2007-12-18 00:00:00,유로4
3,타타대우,대우5톤초장축카고트럭,BC4AT1,DL06,5890.0,270,2500,수동6단,11500,6370,타타대우상용차,한국,8MY-DC-24-12,2008-05-21 00:00:00,유로4
4,타타대우,대우트랙터,EX3TX2,F3BE3681B,12882.0,560,1900,자동12단,26020,8810,타타대우상용차,한국,8MY-DC-24-15,2008-06-30 00:00:00,유로4


In [151]:
l45m = l45[[
    '인증번호',
    '제작사2',
    '차명',
    '제작국',
    '비고'

    # '제작사1',
    # '자동차형식',
    # '엔진형식',
    # '배기량',
    # '최대출력_PS',
    # '최대출력_RPM',
    # '변속기',
    # '총중량',
    # '공차중량',
    # '인증일자',
]]
l45m.head()

,인증번호,제작사2,차명,제작국,비고
0,AMY-HD-14-35,현대자동차,현대25.5톤덤프트럭,한국,유로5
1,7MY-DC-23-70,타타대우상용차,대우25톤장축카고트럭,한국,유로4
2,7MY-DC-23-72,타타대우상용차,대우5톤초장축카고트럭,한국,유로4
3,8MY-DC-24-12,타타대우상용차,대우5톤초장축카고트럭,한국,유로4
4,8MY-DC-24-15,타타대우상용차,대우트랙터,한국,유로4


In [152]:
l45m = l45m.rename(columns={'인증번호':'배출가스인증번호', '제작사2':'제작사', '제작국가':'제작사', '비고':'유럽기준'})
l45m['SCR유무'] = 'Y'
l45m.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,AMY-HD-14-35,현대자동차,현대25.5톤덤프트럭,한국,유로5,Y
1,7MY-DC-23-70,타타대우상용차,대우25톤장축카고트럭,한국,유로4,Y
2,7MY-DC-23-72,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
3,8MY-DC-24-12,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
4,8MY-DC-24-15,타타대우상용차,대우트랙터,한국,유로4,Y


In [153]:
l6.head()

,No,제작사,인증번호,인증일자,차명,차종,차형식,엔진형식,배기량,변속기,최대출력,Unnamed: 11,총중량,공차중량,제작국,사용연료,배출기준,유럽기준,SCR유무
0,6,자일대우버스(주),LMY-DB-14-105,2020-12-31,BX212,승용(초대형),BX212M-GPLBZC441L,OM470LA.6-6,10677,자동화수동변속기,456,1600,16035,13110,한국,경유(Diesel),2017년 10월 기준,유로6,Y
1,8,타타대우상용차(주),LMY-DC-14-87,2020-11-03,LD트럭,화물(대형),FE5BC1,F4AFE411C,4485,수동6단,206,2500,9910,4715,한국,경유(Diesel),2017년 10월 기준,유로6,Y
2,12,타타대우상용차(주),KMY-DC-14-82,2019-11-26,프리마14톤장축카고트럭,화물(초대형),JL8A82,DX12P,11051,자동12단,460,1900,25800(25260),11670(11130),한국,경유(Diesel),2017년 10월 기준,유로6,Y
3,17,(주)우진산전,GMY-WJ-14-02,2016-01-06,우진산전 굴절버스,승용(대형),KM9ZGCZE,ISB6.7E6,6690,없음,280,2100,25995,19495,한국,경유-하이브리드(D-Hybrid),2014년 1월 기준,유로6,Y
4,19,타타대우상용차(주),GMY-DC-14-52,2016-11-07,노부스5톤극초장축플러스8.6카고트럭,화물(대형),DC8CT7,DL06P,5890,수동 9단,280,2500,12500(12710),7370(7580),한국,경유(Diesel),2014년 1월 기준,유로6,Y


In [154]:
l6.columns.to_list()

['No',
 '제작사',
 '인증번호',
 '인증일자',
 '차명',
 '차종',
 '차형식',
 '엔진형식',
 '배기량',
 '변속기',
 '최대출력',
 'Unnamed: 11',
 '총중량',
 '공차중량',
 '제작국',
 '사용연료',
 '배출기준',
 '유럽기준',
 'SCR유무']

In [155]:
l6m = l6[[
    '인증번호',
    '제작사',
    '차명',
    '제작국',
    '유럽기준',
    'SCR유무',
    
    # 'No',
    # '인증일자',
    # '차종',
    # '차형식',
    # '엔진형식',
    # '배기량',
    # '변속기',
    # '최대출력',
    # 'Unnamed: 11',
    # '총중량',
    # '공차중량',
    # '사용연료',
    # '배출기준',
]]
l6m.head()

,인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,LMY-DB-14-105,자일대우버스(주),BX212,한국,유로6,Y
1,LMY-DC-14-87,타타대우상용차(주),LD트럭,한국,유로6,Y
2,KMY-DC-14-82,타타대우상용차(주),프리마14톤장축카고트럭,한국,유로6,Y
3,GMY-WJ-14-02,(주)우진산전,우진산전 굴절버스,한국,유로6,Y
4,GMY-DC-14-52,타타대우상용차(주),노부스5톤극초장축플러스8.6카고트럭,한국,유로6,Y


In [156]:
l6m = l6m.rename(columns={'인증번호':'배출가스인증번호'})
l6m.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,LMY-DB-14-105,자일대우버스(주),BX212,한국,유로6,Y
1,LMY-DC-14-87,타타대우상용차(주),LD트럭,한국,유로6,Y
2,KMY-DC-14-82,타타대우상용차(주),프리마14톤장축카고트럭,한국,유로6,Y
3,GMY-WJ-14-02,(주)우진산전,우진산전 굴절버스,한국,유로6,Y
4,GMY-DC-14-52,타타대우상용차(주),노부스5톤극초장축플러스8.6카고트럭,한국,유로6,Y


In [157]:
s6.head()

,일련번호,제작사,배출가스차종,배출가스인증번호,배출가스인증일,차명,자동차형식,엔진형식,배기량,변속기,최대출력PS,최대출력RPM,총중량,공차중량,제작국가,사용연료,배출가스기준,유럽 기준,SCR 유(Y)무(N)
0,27,현대자동차(주),승용(중형),FMY-HD-14-38,2015-07-29,그랜드스타렉스 2.5디젤 12인(4WD),TQ-25D12-E6-4W5A,D4CB,2497,자동5단,175,3600,3395,2505,한국,경유(Diesel),2014년 1월 기준,유로6,Y
1,29,현대자동차(주),승용(소형),HMY-HD-14-61,2017-12-15,G80 2.2디젤 2WD,DH-22D-H6-2W8A,D4HC,2199,자동8단,202,3800,2435,2025,한국,경유(Diesel),2017년 10월 기준,유로6,Y
2,31,현대자동차(주),승용(소형),HMY-HD-14-63,2017-12-15,G70 2.2 디젤 2WD,IK-22D-E6C-2W8A,D4HC,2199,자동8단,202,3800,2140,1765,한국,경유(Diesel),2017년 10월 기준,유로6,Y
3,32,현대자동차(주),승용(소형),JMY-HD-14-15,2018-02-02,싼타페 2.0디젤 2WD,TM-20D7-H6-2W8A,D4HA,1995,자동8단,186,4000,2485,1910,한국,경유(Diesel),2017년 10월 기준,유로6,Y
4,33,현대자동차(주),승용(소형),HMY-HD-14-62,2017-12-15,G70 2.2 디젤 AWD,IK-22D-E6C-AW8A,D4HC,2199,자동8단,202,3800,2215,1840,한국,경유(Diesel),2017년 10월 기준,유로6,Y


In [158]:
s6.columns.to_list()

['일련번호',
 '제작사',
 '배출가스차종',
 '배출가스인증번호',
 '배출가스인증일',
 '차명',
 '자동차형식',
 '엔진형식',
 '배기량',
 '변속기',
 '최대출력PS',
 '최대출력RPM',
 '총중량',
 '공차중량',
 '제작국가',
 '사용연료',
 '배출가스기준',
 '유럽 기준',
 'SCR 유(Y)무(N)']

In [159]:
s6m = s6[[
    '배출가스인증번호',
    '제작사',
    '차명',
    '제작국가',
    '유럽 기준',
    'SCR 유(Y)무(N)'

    # '일련번호',
    # '배출가스차종',
    # '배출가스인증일',
    # '자동차형식',
    # '엔진형식',
    # '배기량',
    # '변속기',
    # '최대출력PS',
    # '최대출력RPM',
    # '총중량',
    # '공차중량',
    # '사용연료',
    # '배출가스기준',
]]
s6m.head()

,배출가스인증번호,제작사,차명,제작국가,유럽 기준,SCR 유(Y)무(N)
0,FMY-HD-14-38,현대자동차(주),그랜드스타렉스 2.5디젤 12인(4WD),한국,유로6,Y
1,HMY-HD-14-61,현대자동차(주),G80 2.2디젤 2WD,한국,유로6,Y
2,HMY-HD-14-63,현대자동차(주),G70 2.2 디젤 2WD,한국,유로6,Y
3,JMY-HD-14-15,현대자동차(주),싼타페 2.0디젤 2WD,한국,유로6,Y
4,HMY-HD-14-62,현대자동차(주),G70 2.2 디젤 AWD,한국,유로6,Y


In [160]:
s6m = s6m.rename(columns={'SCR 유(Y)무(N)':'SCR유무', '제작국가':'제작국', '유럽 기준':'유럽기준'})
s6m.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,FMY-HD-14-38,현대자동차(주),그랜드스타렉스 2.5디젤 12인(4WD),한국,유로6,Y
1,HMY-HD-14-61,현대자동차(주),G80 2.2디젤 2WD,한국,유로6,Y
2,HMY-HD-14-63,현대자동차(주),G70 2.2 디젤 2WD,한국,유로6,Y
3,JMY-HD-14-15,현대자동차(주),싼타페 2.0디젤 2WD,한국,유로6,Y
4,HMY-HD-14-62,현대자동차(주),G70 2.2 디젤 AWD,한국,유로6,Y


In [161]:
l45m.shape[0] + l6m.shape[0] + s6m.shape[0]

110

In [162]:
cert = pd.concat([l45m, l6m, s6m], ignore_index=True)
cert.shape

(110, 6)

In [163]:
cert.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,AMY-HD-14-35,현대자동차,현대25.5톤덤프트럭,한국,유로5,Y
1,7MY-DC-23-70,타타대우상용차,대우25톤장축카고트럭,한국,유로4,Y
2,7MY-DC-23-72,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
3,8MY-DC-24-12,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
4,8MY-DC-24-15,타타대우상용차,대우트랙터,한국,유로4,Y


### [출력] 인증대장

In [164]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20231208'

In [165]:
# 0.0s
cert.to_csv(os.path.join(raw_fold, f'인증대장_{today_date}.csv'), index=False)

## 등록 + 인증

In [189]:
csm.shape

(9533740, 10)

In [190]:
# 2.0s
cc = csm.merge(cert, on='배출가스인증번호', how='left')
cc.shape

(9533740, 15)

In [191]:
cc.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '차량연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '제작사', '차명', '제작국', '유럽기준', 'SCR유무'],
      dtype='object')

## 등록 + 인증 + 저감

In [192]:
# 13.8s
pr = cc.merge(attm, on='차대번호', how='left')
pr.shape

(9533740, 16)

In [193]:
pr.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '차량연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '제작사', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분'],
      dtype='object')

In [200]:
ex = pr.groupby(['엔진형식', '제작사명'])['차대번호'].count().reset_index()
ex.shape

(8679, 3)

In [203]:
ex.head()

,엔진형식,제작사명,차대번호
0,-,(주) 두성특장차,1
1,-,한국특장차(주),1
2,.D4GB,(주) 골드밴,1
3,0,SOFA차량,4
4,0,남도수입,1


In [204]:
# ex.to_excel(os.path.join(an_fold, '엔진형식_제작사명_리스트.xlsx'), index=False)

## 제작국 설정

In [207]:
pr.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '차량연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '제작사', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분'],
      dtype='object')

In [209]:
ck.columns

Index(['엔진형식', '제작사명', '차대번호', '국내/외 구분(1:국내, 2:국외)'], dtype='object')

In [211]:
pr.shape

(9533740, 16)

In [217]:
pr01 = pr.merge(ck, on=['제작사명', '엔진형식'], how='left')
pr01.shape

(9533740, 17)

In [218]:
pr01.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '차량연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '제작사', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분', '구분'],
      dtype='object')

### [출력] 전처리 완료 파일

In [219]:
# 50.2s
pr01.to_csv(os.path.join(raw_fold, f'등록_인증_저감_국내외구분_{today_date}.csv'), index=False)

## 저공해조치 차량과 SCR차량 구분

In [99]:
# 저공해조치차량
    # 저감장치구분 : 1종+SCR인 차량
# SCR차량(EURO6)
    # SCR유무 : Y
    # 국내 : 2017년 이상
    # 국외 : 2015년 이상

In [223]:
mnt = pr01[pr01['저감장치구분'] == '1종+SCR'].reset_index(drop=True)
mnt.shape

(2372, 17)

### [출력] 저공해조치 차량

In [228]:
mnt.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '차량연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '제작사', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분', '구분'],
      dtype='object')

In [230]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20231211'

In [231]:
# 1.7s
mnt.to_excel(os.path.join(df_fold, f'경유_저공해조치차량_{today_date}.xlsx'), index=False)

In [224]:
pr01.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '차량연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '제작사', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분', '구분'],
      dtype='object')

In [225]:
pr01['차량연식'].dtype

dtype('int64')

In [227]:
euro6 = pr01[(pr01['저감장치구분'] != '1종+SCR') & ( ((pr01['구분'] == '국내') & (pr01['차량연식'] >= 2017)) | ((pr01['구분'] == '국외') & (pr01['차량연식'] >= 2015)) | (pr['SCR유무'] == 'Y') )].reset_index(drop=True)
euro6.shape

(4433293, 17)

In [237]:
div = 50 * 10**4
for n in range(euro6.shape[0] // div + 1):
    print(n+1, n*div, (n+1) *div)

1 0 500000
2 500000 1000000
3 1000000 1500000
4 1500000 2000000
5 2000000 2500000
6 2500000 3000000
7 3000000 3500000
8 3500000 4000000
9 4000000 4500000


In [238]:
# 
div = 50 * 10**4
for n in range(euro6.shape[0] // div + 1):
    # print(n*div, (n+1) *div)
    euro6.iloc[n*div:(n+1)*div].to_excel(os.path.join(df_fold, f'EURO6차량현황{n+1}_{today_date}.xlsx'), index=False)